Referenced from [Langchain](https://python.langchain.com/docs/tutorials/rag/); this was enough of a RAG to figure out where the pain points are.

In [ ]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma
%pip install --quiet --upgrade pypdf #TODO PyPDF prob doesn't have native code/is slow.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
# TODO: There's a bunch of table reqs in this format; probably screws stuff up.
loader = PyPDFLoader("req_docs/20190029153.pdf") # GATEWAY SYSTEM REQUIREMENTS
docs = loader.load_and_split() # Split based on page breaks.

In [ ]:
# See, the tables screw things up.
print(docs[4])

Document too large, split it into chunks with overlap.

> We use the `RecursiveCharacterTextSplitter`, which will recursively split the document using common separators like new lines until each chunk is the appropriate size.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

CHUNK_SIZE=1000
CHUNK_OVERLAP=200

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP, add_start_index=True
)

all_splits = text_splitter.split_documents(docs)
len(all_splits)

It seems there are `splitters` oriented toward specific types of documents: there are loaders for "source code" and "academic dissertations".

Perhaps write something for Requirements Documents... ?

[MBET Embedding Leaderboard](https://huggingface.co/spaces/mteb/leaderboard)

In [ ]:
# %pip install --quiet --upgrade langchain_voyageai
# %pip install --quiet --upgrade langchain_openai
# %pip install -qU langchain_ollama
%pip install -qU langchain-huggingface

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# from langchain_voyageai import VoyageAIEmbeddings
# from langchain_openai import OpenAIEmbeddings

# embeddings = VoyageAIEmbeddings(
#    voyage_api_key="paChL4fTAr1Rbb9YcJ1Rmo", model="voyage-2"
# )

# local_embeddings = OllamaEmbeddings(model="nomic-embed-text")
# print(local_embeddings)
# TODO: I've either got to pay $$$ for embeddings for a provider, or fetch a computer that can run ollama! :-)

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print(embeddings_model)

# I'm going to slice the splits to just 1 at this point for PoC.
# vectorstore = Chroma.from_documents(documents=all_splits[100], embedding=embeddings_model)

In [ ]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings_model)

In [ ]:
question = "Active and Passive thermal systems should be designed with enough robustness and capacity to allow Gateway to fly in attitudes that are driven/support operations for extended periods."
sim = vectorstore.similarity_search(question)
print(len(sim))

In [ ]:
print(sim[2].page_content)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# TODO: Integrate the prompts.. what are we supposed to ask here?
# TODO: Where's my inference?